##  Data Preprocessing

This notebook prepares the dataset for modeling by handling missing values, encoding categorical features, and scaling numerical variables where appropriate.

### Goals:
- Clean missing or invalid data
- Standardize feature formats (numerical/categorical)
- Apply encoding and scaling techniques as needed
- Output a modeling-ready dataset
---

In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))
from scripts.data_loader import load_raw

In [3]:
# Load the dataset
df = load_raw("data-corrupted.csv")


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,NaN,...,4,3,4,1.0,1,3,6,5,6.0,6
1,NaN,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1.0,1,3,4,5,5.0,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2.0,3,3,10,7,8.0,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1.0,1,5,2,15,14.0,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1.0,2,5,4,6,10.0,10
